# Python for Operations Research

## Pyomo - Intermediate

**Mohammad Daneshvar - Ph.D. Candidate**

ESG-UQAM

# What is Pyomo?
- Set of software packages
- Formulate optimization problems
- Doesn't solve them

# What are the other options?
- Gurobi Python Interface: [gurobipy](https://pypi.org/project/gurobipy/)
- Cplex Python Interface: [cplex](https://pypi.org/project/cplex/) or [docplex](https://pypi.org/project/docplex/)
- GNU Linear Programming Kit Python Interface: [glpk](https://pypi.org/project/glpk/)



# Pyomo Overview

In [1]:
import pyomo.environ as pyo

# Study Problem
- I need to buy a new set of t-shirts for summer
- My budget is 100\$
- These are my options:
    - red t-shirt: 57\$
    - blue t-shirt: 44\$
    - green t-shirt: 44\$
    - black t-shirt: 40\$

**Selection Criteria**
- How much I like each t-shirt! (t-shirt value)
    - red t-shirt: 80
    - blue t-shirt: 80
    - green t-shirt: 85
    - black t-shirt: 40

# A Simple Concrete Pyomo Model

$$max\;\;\; 80x_{blue} + 85x_{green} + 80x_{red} + 40x_{black}$$
$$s.t.\;\; 44x_{blue} + 44x_{green} + 57x_{red} + 40x_{black} \leq 100 $$
$$x_{blue}, x_{green}, x_{red}, x_{black}\in \{0, 1\}$$

In [2]:
model = pyo.ConcreteModel()

model.x = pyo.Var(['blue','green', 'red', 'black'], domain=pyo.Boolean)

model.objective = pyo.Objective(expr = 80*model.x['blue'] + 85*model.x['green']
                                + 80*model.x['red'] + 40*model.x['black'],
                                sense = pyo.maximize)

model.budget_constraint = pyo.Constraint(expr = 44*model.x['blue'] + 44*model.x['green']
                                         + 57*model.x['red'] + 40*model.x['black']
                                         <= 100)

# A Simple Abstract Pyomo Model

$$\max \sum\limits_{j=1}^nv_jx_j$$
$$s.t.\;\sum\limits_{j=1}^nc_{j}x_j\leq b$$
$$x_j\in  \{0,1\}\;\;\;\forall j = 1...n$$

In [3]:
model = pyo.AbstractModel()

model.n = pyo.Param(within=pyo.NonNegativeIntegers)  # number of colors

model.J = pyo.RangeSet(1, model.n)  # set of all color indices

model.c = pyo.Param(model.J)  # cost of each item
model.b = pyo.Param()  # budget
model.v = pyo.Param(model.J)  # value of each item

model.x = pyo.Var(model.J, domain=pyo.NonNegativeReals)

def obj_expression(m):
    return pyo.summation(m.v, m.x)

model.objective = pyo.Objective(rule=obj_expression)

def ax_constraint_rule(m):
    return sum(m.c[j] * m.x[j] for j in m.J) <= m.b

model.budget_constraint = pyo.Constraint(rule=ax_constraint_rule)

- It is a mathematical model: represent a set of mathematical problems
- Does not depend on the data

# Concrete or Abstract?
- These are original ideas by the Pyomo developers.
- Over the years users have shown more interest in the concrete model.

# Pyomo Modeling Components
- Sets
- Parameters
- Variables
- Objectives
- Constraints
- Expressions
- Suffixes

# Sets
- Declaration
- Operations
    - Union: |
    - Intersection: &
    - Difference: -
    - Exclusive or: ^

In [4]:
m = pyo.ConcreteModel()
m.set_of_warm_colors = pyo.Set(initialize=['red'])
m.set_of_cool_colors = pyo.Set(initialize=['green', 'blue'])
m.set_of_neutral_colors = pyo.Set(initialize=['black'])
m.set_all_colors =  m.set_of_warm_colors | m.set_of_cool_colors | m.set_of_neutral_colors
m.set_all_colors.pprint()

set_all_colors : Size=1, Index=None, Ordered=True
    Key  : Dimen : Domain                                         : Size : Members
    None :     1 : set_all_colors_index_0 | set_of_neutral_colors :    4 : {'red', 'green', 'blue', 'black'}


In [5]:
m.set_intersection_exmaple = m.set_all_colors & m.set_of_warm_colors
m.set_intersection_exmaple.pprint()

set_intersection_exmaple : Size=1, Index=None, Ordered=True
    Key  : Dimen : Domain                              : Size : Members
    None :     1 : set_all_colors & set_of_warm_colors :    1 : {'red',}


# Parameters
- Declaration

In [6]:
def price_initializer(m, color):
    if color == "black":
        return 40
    elif any(color in colors for colors in m.set_of_cool_colors):
        return 44
    elif color == "red":
        return 57
    return -1

In [7]:
m.color_price = pyo.Param(m.set_all_colors, initialize=price_initializer,
                          within=pyo.NonNegativeReals)
m.color_price.pprint()

value_dict = {'black': 40, 'blue': 80, 'green': 85, 'red': 80}

m.color_value = pyo.Param(m.set_all_colors, initialize=value_dict, within=pyo.NonNegativeReals)
m.color_value.pprint()

m.budget = pyo.Param(initialize=100)
m.budget.pprint()

color_price : Size=4, Index=set_all_colors, Domain=NonNegativeReals, Default=None, Mutable=False
    Key   : Value
    black :    40
     blue :    44
    green :    44
      red :    57
color_value : Size=4, Index=set_all_colors, Domain=NonNegativeReals, Default=None, Mutable=False
    Key   : Value
    black :    40
     blue :    80
    green :    85
      red :    80
budget : Size=1, Index=None, Domain=Any, Default=None, Mutable=False
    Key  : Value
    None :   100


# Variables
    - Bounds
    - Domain
    - Initialize

In [8]:
m.selected_color = pyo.Var(m.set_all_colors, domain=pyo.Boolean)
m.selected_color.pprint()

selected_color : Size=4, Index=set_all_colors
    Key   : Lower : Value : Upper : Fixed : Stale : Domain
    black :     0 :  None :     1 : False :  True : Boolean
     blue :     0 :  None :     1 : False :  True : Boolean
    green :     0 :  None :     1 : False :  True : Boolean
      red :     0 :  None :     1 : False :  True : Boolean


# Objectives

In [9]:
def objecitve_function_rule(m):
    return sum(m.selected_color[i] * m.color_value[i] for i in m.set_all_colors)

In [10]:
m.objective_function = pyo.Objective(rule=objecitve_function_rule, sense=pyo.maximize)
m.objective_function.pprint()

objective_function : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : maximize : 80*selected_color[red] + 85*selected_color[green] + 80*selected_color[blue] + 40*selected_color[black]


# Constraints

In [11]:
def color_rule(m):
    return sum(m.selected_color[i]*m.color_price[i] for i in m.set_all_colors) <= m.budget

In [12]:
m.color_constraint = pyo.Constraint(rule=color_rule)
m.color_constraint.pprint()

color_constraint : Size=1, Index=None, Active=True
    Key  : Lower : Body                                                                                                   : Upper : Active
    None :  -Inf : 57*selected_color[red] + 44*selected_color[green] + 44*selected_color[blue] + 40*selected_color[black] : 100.0 :   True


# Solve the Model 

In [13]:
from pyomo.core import *

opt = pyomo.environ.SolverFactory('gurobi')

result = opt.solve(m, options={'TimeLimit': 100})

m.selected_color.pprint()

selected_color : Size=4, Index=set_all_colors
    Key   : Lower : Value : Upper : Fixed : Stale : Domain
    black :     0 :   0.0 :     1 : False : False : Boolean
     blue :     0 :   1.0 :     1 : False : False : Boolean
    green :     0 :   1.0 :     1 : False : False : Boolean
      red :     0 :   0.0 :     1 : False : False : Boolean


# Repeated Solves

In [14]:
m.cuts = pyo.ConstraintList()

for i in range(2):
   expr = 0
   for j in m.selected_color:
       if pyo.value(m.selected_color[j]) < 0.5:
           expr += m.selected_color[j]
       else:
           expr += (1 - m.selected_color[j])
   m.cuts.add( expr >= 1 )
   results = opt.solve(m)
   print ("\n===== iteration",i)
   m.selected_color.pprint()


===== iteration 0
selected_color : Size=4, Index=set_all_colors
    Key   : Lower : Value : Upper : Fixed : Stale : Domain
    black :     0 :   1.0 :     1 : False : False : Boolean
     blue :     0 :  -0.0 :     1 : False : False : Boolean
    green :     0 :   1.0 :     1 : False : False : Boolean
      red :     0 :  -0.0 :     1 : False : False : Boolean

===== iteration 1
selected_color : Size=4, Index=set_all_colors
    Key   : Lower : Value : Upper : Fixed : Stale : Domain
    black :     0 :   1.0 :     1 : False : False : Boolean
     blue :     0 :   0.0 :     1 : False : False : Boolean
    green :     0 :   0.0 :     1 : False : False : Boolean
      red :     0 :   1.0 :     1 : False : False : Boolean


# Stochastic Programming

In [15]:
del m.cuts  

del m.objective_function

del m.color_value

In [16]:
m.set_scenarios = pyo.Set(initialize=['happy', 'normal', 'sad'])

def st_value_set(m, scenario_name, color_name):
    scenario_dict = {'happy' : {'black': 40, 'blue': 80, 'green': 88, 'red': 100},
                     'normal' : {'black': 40, 'blue': 80, 'green': 85, 'red': 80},
                     'sad' : {'black': 100, 'blue': 80, 'green': 10, 'red': 80}}
    return scenario_dict[scenario_name][color_name]

m.st_color_value = Param(m.set_scenarios, m.set_all_colors, initialize=st_value_set,
                         within=pyo.NonNegativeReals)

$$\max \sum\limits_{s=1}^{3}p_s\sum\limits_{j=1}^n v_{sj}x_j$$
$$s.t.\;\sum\limits_{j=1}^nc_{j}x_j\leq b$$
$$x_j\in  \{0,1\}\;\;\;\forall j = 1...n$$

In [17]:
def st_objecitve_function_rule(m):
    return (1.0/3) * sum(m.selected_color[i] * m.st_color_value[j, i] 
                         for i in m.set_all_colors for j in m.set_scenarios)

m.objective_function = pyo.Objective(rule=st_objecitve_function_rule, sense=pyo.maximize)

In [18]:
result = opt.solve(m, options={'TimeLimit': 100})

m.selected_color.pprint()

selected_color : Size=4, Index=set_all_colors
    Key   : Lower : Value : Upper : Fixed : Stale : Domain
    black :     0 :   1.0 :     1 : False : False : Boolean
     blue :     0 :   0.0 :     1 : False : False : Boolean
    green :     0 :   0.0 :     1 : False : False : Boolean
      red :     0 :   1.0 :     1 : False : False : Boolean


# Thank you! 
